In [1]:
import guitarpro as gp
import mgzip
import pickle
import copy

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from torch import nn
import numpy as np

from dataset import Note, NotesDataset
from model import CustomTransformer
from training import train
from inference import inference_model, decompose_note, split_by_measures
from downloading import Song

/home/natalia/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# Loading data

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(0)
MIDI  = {str(gp.GuitarString(number=0, value=val)) : val for val in range(128)}
standard_tuning = {1: MIDI['E4'], 2: MIDI['B3'], 3: MIDI['G3'], 4: MIDI['D3'], 5: MIDI['A2'], 6: MIDI['E2']}

In [3]:
with mgzip.open('notes_data.pickle.gz', 'rb') as filepath:
    notes = pickle.load(filepath)
    note_to_int = pickle.load(filepath)
    int_to_note = pickle.load(filepath)
    n_vocab = pickle.load(filepath)

In [4]:
int_notes = np.array([note_to_int[note] for note in notes])
seq_len = 100
idx = seq_len * ((len(int_notes) - 1) // seq_len)

In [5]:
x_train, y_train = int_notes[:idx].reshape(-1, seq_len), int_notes[1:idx+1].reshape(-1, seq_len)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=0, test_size=0.1)

In [6]:
bs = 512
train_dataset = NotesDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, bs, shuffle=False)
val_dataset = NotesDataset(x_val, y_val)
val_loader = DataLoader(val_dataset, bs, shuffle=False)

# Model

In [7]:
n_semitones = int(int_to_note[n_vocab-1][0]) + 1 # Include 0
n_durations = int(np.log2(max([v[1] for v in int_to_note.values()]))) + 1
emb_dim = n_semitones + 3 + n_durations + 2

In [8]:
init_embeddings = np.zeros((n_vocab, emb_dim))
for i, k in enumerate(int_to_note.keys()):
    cur_note = Note(int_to_note[k][0], int_to_note[k][1], int_to_note[k][2], int_to_note[k][3])
    init_embeddings[i] = cur_note.vectorize(n_semitones, n_durations)
init_embeddings = (torch.tensor(init_embeddings) - init_embeddings.mean()).to(device)

In [9]:
model = CustomTransformer(bs, n_vocab, emb_dim, seq_len, init_embeddings).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003, betas=(0.96, 0.999))

# Training and generation

In [ ]:
train(model, train_loader, val_loader, criterion, optimizer, 200)

Validating 1/200: 100%|██████████| 2/2 [00:00<00:00,  5.97it/s]


Best model ever
Epoch: 1, Train loss: 5.432, Val loss: 5.343


Validating 2/200: 100%|██████████| 2/2 [00:00<00:00,  6.04it/s]


Best model ever
Epoch: 2, Train loss: 5.150, Val loss: 5.075


Validating 3/200: 100%|██████████| 2/2 [00:00<00:00,  5.93it/s]


...

Validating 121/200: 100%|██████████| 2/2 [00:00<00:00,  5.78it/s]


Epoch: 121, Train loss: 2.459, Val loss: 2.847


Validating 122/200: 100%|██████████| 2/2 [00:00<00:00,  6.01it/s]


Epoch: 122, Train loss: 2.444, Val loss: 2.799


Validating 123/200: 100%|██████████| 2/2 [00:00<00:00,  5.07it/s]


In [10]:
with mgzip.open("track_data.pickle.gz", 'rb') as filepath:
    track_data = pickle.load(filepath)

In [11]:
np.random.seed(42)

track_idx1 = 58
first_note_idx1 = track_data.loc[track_idx1]["noteStartIdx"]
last_note_idx1 = track_data.loc[track_idx1 + 1]["noteStartIdx"]
gap = np.random.randint(0, last_note_idx1 - first_note_idx1)
start_note_idx1 = first_note_idx1 + gap

len_ = np.random.choice(np.arange(min(100, last_note_idx1 - first_note_idx1)))
start_seq = [val for val in int_notes[start_note_idx1:start_note_idx1+len_]]
new_seq1 = inference_model(model, start_seq, int_to_note)

/home/natalia/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [13]:
track_idx2 = 59
first_note_idx2 = track_data.loc[track_idx2]["noteStartIdx"]
last_note_idx2 = track_data.loc[track_idx2 + 1]["noteStartIdx"]
start_note_idx2 = first_note_idx2 + gap

start_seq = [val for val in int_notes[start_note_idx2:start_note_idx2+len_]]
new_seq2 = inference_model(model, start_seq, int_to_note)

# Postprocessing

In [14]:
new_track1 = split_by_measures(new_seq1)
new_track2 = split_by_measures(new_seq2)

In [15]:
if len(new_track1) < len(new_track2):
    track = new_track1.copy()
    new_track1 = new_track2
    new_track2 = track

    track_idx = copy.deepcopy(track_idx1)
    track_idx1 = track_idx2
    track_idx2 = track_idx

In [16]:
def tune_track(idx):
    lowest = track_data.loc[idx]['tuning']
    tuning = standard_tuning.copy()
    if lowest <= MIDI['B1']:
        tuning[7] = MIDI['B1']
        decrease = MIDI['B1'] - lowest
    else:
        decrease = MIDI['E2'] - lowest
    return {k: v - decrease for k, v in tuning.items()}

new_track_tuning1 = tune_track(track_idx1)
new_track_tuning2 = tune_track(track_idx2)

# Downloading the track

In [ ]:
tempo = track_data.loc[track_idx1]['tempo']
instrument = track_data.loc[track_idx1]['instrument']
name = track_data.loc[track_idx1]['song']

songWriter = Song(tempo)
songWriter.process_track(new_track1, new_track_tuning1, instrument=instrument, name=name)

In [67]:
tempo = track_data.loc[track_idx2]['tempo']
instrument = track_data.loc[track_idx2]['instrument']
name = track_data.loc[track_idx2]['song']

songWriter.process_track(new_track2, new_track_tuning2, instrument=instrument, name=name)

In [68]:
songWriter.write('generation58.gp5')